In [1]:
import networkx as nx
import time
import pickle

from heuristic_custom import HeuristicCustom
from heuristic_rvsa import HeuristicRVSA
from heuristic_obj_max import ObjectiveFunctionMaximizer
from exact_gurobi import ExactSolutionGurobi

# Configure the parameters of the Framework

In [2]:
# Create an undirected graph for the substrate network
substrate_network = nx.Graph()

# Add nodes with physical capacities and availability
substrate_network.add_node('S', capacity={'cpu': 10, 'memory': 20}, availability=1)
substrate_network.add_node('A', capacity={'cpu': 10, 'memory': 20}, availability=0.99)
substrate_network.add_node('B', capacity={'cpu': 10, 'memory': 20}, availability=0.85)
substrate_network.add_node('C', capacity={'cpu': 10, 'memory': 20}, availability=0.98)
substrate_network.add_node('G', capacity={'cpu': 10, 'memory': 20}, availability=0.99)
substrate_network.add_node('D', capacity={'cpu': 10, 'memory': 20}, availability=1)

# Add edges with bandwidth, delay, and availability
substrate_network.add_edge('S', 'A', bandwidth=10, delay=10, availability=0.9)
substrate_network.add_edge('S', 'C', bandwidth=10, delay=10, availability=0.95)
substrate_network.add_edge('A', 'B', bandwidth=10, delay=10, availability=0.8)
substrate_network.add_edge('A', 'G', bandwidth=10, delay=10, availability=0.7)
substrate_network.add_edge('A', 'C', bandwidth=10, delay=10, availability=0.6)
substrate_network.add_edge('C', 'G', bandwidth=10, delay=10, availability=0.75)
substrate_network.add_edge('C', 'B', bandwidth=10, delay=10, availability=0.85)
substrate_network.add_edge('B', 'G', bandwidth=10, delay=10, availability=0.5)
substrate_network.add_edge('B', 'D', bandwidth=10, delay=10, availability=0.95)
substrate_network.add_edge('G', 'D', bandwidth=10, delay=10, availability=0.88)

# Example slice request 1
slice_request_1 = nx.DiGraph()
slice_request_1.add_node('S1', demand={'cpu': 0, 'memory': 0})
slice_request_1.add_node('F1', demand={'cpu': 2, 'memory': 2})
slice_request_1.add_node('F2', demand={'cpu': 5, 'memory': 5})
slice_request_1.add_node('D1', demand={'cpu': 0, 'memory': 0})
slice_request_1.add_edge('S1', 'F1', bandwidth=3)
slice_request_1.add_edge('F1', 'F2', bandwidth=3)
slice_request_1.add_edge('F2', 'D1', bandwidth=3)
slice_request_1.add_edge('F1', 'D1', bandwidth=3)
slice_request_1.graph['max_delay'] = 50  # Maximum end-to-end delay requirement for slice 1
slice_request_1.graph['min_availability'] = 0.1  # Minimum availability requirement for slice 1
slice_request_1.graph['max_placement_groups'] = 1  # K_i for slice 1
slice_request_1.graph['node_s_mapping'] = ('S1', 'S') # S1 should be mapped to S
slice_request_1.graph['node_d_mapping'] = ('D1', 'D') # D1 should be mapped to D

# Example slice request 2
slice_request_2 = nx.DiGraph()
slice_request_2.add_node('S2', demand={'cpu': 0, 'memory': 0})
slice_request_2.add_node('VNF1', demand={'cpu': 2, 'memory': 3})
slice_request_2.add_node('VNF2', demand={'cpu': 5, 'memory': 4})
slice_request_2.add_node('D2', demand={'cpu': 0, 'memory': 0})
slice_request_2.add_edge('S2', 'VNF1', bandwidth=3)
slice_request_2.add_edge('VNF1', 'VNF2', bandwidth=3)
slice_request_2.add_edge('VNF2', 'D2', bandwidth=3)
slice_request_2.graph['max_delay'] = 60  # Maximum end-to-end delay requirement for slice 2
slice_request_2.graph['min_availability'] = 0.75  # Minimum availability requirement for slice 2
slice_request_2.graph['max_placement_groups'] = 2  # K_i for slice 2
slice_request_2.graph['node_s_mapping'] = ('S2', 'S') # S1 should be mapped to S
slice_request_2.graph['node_d_mapping'] = ('D2', 'D') # D1 should be mapped to D


# Create a dictionary of slice requests
slice_requests = {1: slice_request_1, 2: slice_request_2}

object_coeffs = {
    # Coefficients for the objective function
    'rho1' : 1, 'rho2' : 1, 'rho3' : 1,
    # Coefficients for QoE function coefficients (per resources per slice request)
    'beta' : {(1, 'cpu') : 1.8, (1, 'memory') : 1.8, (1, 'delay') : 1, (1, 'bw') : 1.8,
              (2, 'cpu') : 1.8, (2, 'memory') : 1.8, (2, 'delay') : 1, (2, 'bw') : 1.8},
    'gamma' : {(1, 'cpu') : 1, (1, 'memory') : 1, (1, 'delay') : -3.9, (1, 'bw') : 1,
               (2, 'cpu') : 1, (2, 'memory') : 1, (2, 'delay') : -3.9, (2, 'bw') : 1},
    'lambda' : {(1, 'cpu') : -0.93, (1, 'memory') : -0.93, (1, 'delay') : 1.6, (1, 'bw') : -0.93,
                (2, 'cpu') : -0.93, (2, 'memory') : -0.93, (2, 'delay') : 1.6, (2, 'bw') : -0.93},
    'mu' : {(1, 'cpu') : 4.8, (1, 'memory') : 4.8, (1, 'delay') : -0.1, (1, 'bw') : 4.8,
            (2, 'cpu') : 4.8, (2, 'memory') : 4.8, (2, 'delay') : -0.1, (2, 'bw') : 4.8},
}

# Run Solutions

## Run Exact Solution using Gurobi

In [3]:
# Create Solver object
exact_solution_obj = ExactSolutionGurobi(G_sub=substrate_network, 
                                         SFC_dict=slice_requests, 
                                         obj_coeffs=object_coeffs,
                                         log_file_path=f"example_run_exact_model_logs.log")
# Set parameters of the Gurobi model
param_dict = {'MIQCPMethod': 1, 'MIPFocus': 2, 'NonConvex': -1, 
              'PreMIQCPForm': 1, 'PreQLinearize': 0, 'PSDTol': 1e-06, 
              'Method': 2, 'Presolve': 2, 'Seed' : 3, 'Cuts': 3, 
              'NodeMethod': 1, "TimeLimit" : 60}
for k, v in param_dict.items():
    exact_solution_obj.model.setParam(k, v)


Set parameter WLSAccessID
Set parameter WLSSecret
Set parameter LicenseID to value 2520987
Academic license 2520987 - for non-commercial use only - registered to do___@tmit.bme.hu
Set parameter LogFile to value "example_run_exact_model_logs.log"
Set parameter MIQCPMethod to value 1
Set parameter MIPFocus to value 2
Set parameter PreMIQCPForm to value 1
Set parameter PreQLinearize to value 0
Set parameter Method to value 2
Set parameter Presolve to value 2
Set parameter Seed to value 3
Set parameter Cuts to value 3
Set parameter NodeMethod to value 1
Set parameter TimeLimit to value 60


In [4]:
# Run the optimization
exact_solution_obj.optimize()

Gurobi Optimizer version 11.0.2 build v11.0.2rc0 (linux64 - "Ubuntu 22.04.4 LTS")

CPU model: AMD Ryzen 5 3600 6-Core Processor, instruction set [SSE2|AVX|AVX2]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Academic license 2520987 - for non-commercial use only - registered to do___@tmit.bme.hu
Optimize a model with 585 rows, 1230 columns and 2148 nonzeros
Model fingerprint: 0xf982d54e
Model has 2 quadratic objective terms
Model has 308 quadratic constraints
Model has 408 general constraints
Variable types: 779 continuous, 451 integer (449 binary)
Coefficient statistics:
  Matrix range     [1e-02, 1e+01]
  QMatrix range    [1e+00, 1e+00]
  QLMatrix range   [1e+00, 1e+00]
  Objective range  [1e+00, 1e+00]
  QObjective range [2e+00, 2e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 6e+01]
  QRHS range       [1e-01, 1e+00]
  PWLCon x range   [0e+00, 2e+00]
  PWLCon y range   [0e+00, 5e+00]
  GenCon rhs range [2e+00, 5e+00]
  GenCon coe rang

In [5]:
# Save the results
exact_solution_obj.save(f"example_run_exact")

## Run Baseline Heuristic RVSA

In [6]:
# Create Solver object
rvsa_heur_obj = HeuristicRVSA(G_sub=substrate_network, SFC_dict=slice_requests)

In [7]:
# 1. Run the heuristic to find a feasible placement
heuristic_placement_time_st = time.perf_counter()
placement = rvsa_heur_obj.run()
heuristic_placement_time_end = time.perf_counter()
heuristic_placement_time = heuristic_placement_time_end - heuristic_placement_time_st
print(f"RVSA Heuristic runtime: {heuristic_placement_time:.4f} sec")

RVSA Heuristic runtime: 0.0174 sec


In [8]:
# 2. Check if the placement was successful
if placement is not None:
    # We found a feasible placement
    # Create the object which allocate more resources to improve QoE --> improve objective function
    rvsa_obj_func_max = ObjectiveFunctionMaximizer(G_sub=substrate_network, SFC_dict=slice_requests, 
                                              obj_coeffs=object_coeffs, placement=placement)
    obj_max_st = time.perf_counter()
    rvsa_obj_func_max.run()
    obj_max_end = time.perf_counter()
    obj_max_time = obj_max_end - obj_max_st

    # Save the intermediate data
    final_solution = rvsa_obj_func_max.get_final_solution()
    objective_scores = rvsa_obj_func_max.get_objective_scores()
    with open(f'example_run_rvsa_solution.pickle', 'wb') as f:
        pickle.dump(final_solution, f)
    total_time = heuristic_placement_time + obj_max_time
    print(f"Solution found. Placement time = {heuristic_placement_time:.4f}, Objective score maximization time = {obj_max_time:.4f}, total = {total_time:.4f}")
    print(f"Objective scores: {objective_scores}")
else:
    # We did not find a feasible solution, we conclude that the problem is infeasible
    print("No solution found.")

/home/jovyan/work/BME/qoe_and_availability_aware_slice_placement/heuristic_obj_max.py:147: RuntimeWarning: invalid value encountered in log
  qoe_val = beta * np.log(gamma * qos_val + lambd) + mu


Solution found. Placement time = 0.0174, Objective score maximization time = 1.0990, total = 1.1164
Objective scores: {'objective_value': -172.56004536080525, 'objective_1_value': 31.52664950669189, 'objective_2_value': 189.08669486749713, 'objective_3_value': 12, 'objective_4_value': 3}


## Run ResEff Heuristic

In [9]:
# Create Solver object
reseff_heur_obj = HeuristicCustom(G_sub=substrate_network, SFC_dict=slice_requests, method="resource_saving")

In [10]:
# 1. Run the heuristic to find a feasible placement
heuristic_placement_time_st = time.perf_counter()
placement = reseff_heur_obj.run()
heuristic_placement_time_end = time.perf_counter()
heuristic_placement_time = heuristic_placement_time_end - heuristic_placement_time_st
print(f"ResEff Heuristic runtime: {heuristic_placement_time:.4f} sec")

ResEff Heuristic runtime: 0.0309 sec


In [11]:
# 2. Check if the placement was successful
if placement is not None:
    # We found a feasible placement
    # Create the object which allocate more resources to improve QoE --> improve objective function
    reseff_obj_func_max = ObjectiveFunctionMaximizer(G_sub=substrate_network, SFC_dict=slice_requests, 
                                              obj_coeffs=object_coeffs, placement=placement)
    obj_max_st = time.perf_counter()
    reseff_obj_func_max.run()
    obj_max_end = time.perf_counter()
    obj_max_time = obj_max_end - obj_max_st

    # Save the intermediate data
    final_solution = reseff_obj_func_max.get_final_solution()
    objective_scores = reseff_obj_func_max.get_objective_scores()
    with open(f'example_run_reseff_solution.pickle', 'wb') as f:
        pickle.dump(final_solution, f)
    total_time = heuristic_placement_time + obj_max_time
    print(f"Solution found. Placement time = {heuristic_placement_time:.4f}, Objective score maximization time = {obj_max_time:.4f}, total = {total_time:.4f}")
    print(f"Objective scores: {objective_scores}")
else:
    # We did not find a feasible solution, we conclude that the problem is infeasible
    print("No solution found.")

Solution found. Placement time = 0.0309, Objective score maximization time = 1.0727, total = 1.1036
Objective scores: {'objective_value': -157.37286005656924, 'objective_1_value': 32.318581107258616, 'objective_2_value': 176.69144116382785, 'objective_3_value': 10, 'objective_4_value': 3}


## Run EdgeDisj Heuristic

In [12]:
# Create Solver object
edgedisj_heur_obj = HeuristicCustom(G_sub=substrate_network, SFC_dict=slice_requests, method="link_disjoint")

In [13]:
# 1. Run the heuristic to find a feasible placement
heuristic_placement_time_st = time.perf_counter()
placement = edgedisj_heur_obj.run()
heuristic_placement_time_end = time.perf_counter()
heuristic_placement_time = heuristic_placement_time_end - heuristic_placement_time_st
print(f"ResEff Heuristic runtime: {heuristic_placement_time:.4f} sec")

ResEff Heuristic runtime: 0.0352 sec


In [14]:
# 2. Check if the placement was successful
if placement is not None:
    # We found a feasible placement
    # Create the object which allocate more resources to improve QoE --> improve objective function
    edgedisj_obj_func_max = ObjectiveFunctionMaximizer(G_sub=substrate_network, SFC_dict=slice_requests, 
                                              obj_coeffs=object_coeffs, placement=placement)
    obj_max_st = time.perf_counter()
    edgedisj_obj_func_max.run()
    obj_max_end = time.perf_counter()
    obj_max_time = obj_max_end - obj_max_st

    # Save the intermediate data
    final_solution = edgedisj_obj_func_max.get_final_solution()
    objective_scores = edgedisj_obj_func_max.get_objective_scores()
    with open(f'example_run_edgedisj_solution.pickle', 'wb') as f:
        pickle.dump(final_solution, f)
    total_time = heuristic_placement_time + obj_max_time
    print(f"Solution found. Placement time = {heuristic_placement_time:.4f}, Objective score maximization time = {obj_max_time:.4f}, total = {total_time:.4f}")
    print(f"Objective scores: {objective_scores}")
else:
    # We did not find a feasible solution, we conclude that the problem is infeasible
    print("No solution found.")

Solution found. Placement time = 0.0352, Objective score maximization time = 1.0696, total = 1.1048
Objective scores: {'objective_value': -190.7054729019009, 'objective_1_value': 32.73992107636443, 'objective_2_value': 208.44539397826532, 'objective_3_value': 12, 'objective_4_value': 3}


## Run NodeDisj Heuristic

In [15]:
# Create Solver object
nodedisj_heur_obj = HeuristicCustom(G_sub=substrate_network, SFC_dict=slice_requests, method="node_disjoint")

In [16]:
# 1. Run the heuristic to find a feasible placement
heuristic_placement_time_st = time.perf_counter()
placement = nodedisj_heur_obj.run()
heuristic_placement_time_end = time.perf_counter()
heuristic_placement_time = heuristic_placement_time_end - heuristic_placement_time_st
print(f"ResEff Heuristic runtime: {heuristic_placement_time:.4f} sec")

ResEff Heuristic runtime: 0.0340 sec


In [17]:
# 2. Check if the placement was successful
if placement is not None:
    # We found a feasible placement
    # Create the object which allocate more resources to improve QoE --> improve objective function
    nodedisj_obj_func_max = ObjectiveFunctionMaximizer(G_sub=substrate_network, SFC_dict=slice_requests, 
                                              obj_coeffs=object_coeffs, placement=placement)
    obj_max_st = time.perf_counter()
    nodedisj_obj_func_max.run()
    obj_max_end = time.perf_counter()
    obj_max_time = obj_max_end - obj_max_st

    # Save the intermediate data
    final_solution = nodedisj_obj_func_max.get_final_solution()
    objective_scores = nodedisj_obj_func_max.get_objective_scores()
    with open(f'example_run_nodedisj_solution.pickle', 'wb') as f:
        pickle.dump(final_solution, f)
    total_time = heuristic_placement_time + obj_max_time
    print(f"Solution found. Placement time = {heuristic_placement_time:.4f}, Objective score maximization time = {obj_max_time:.4f}, total = {total_time:.4f}")
    print(f"Objective scores: {objective_scores}")
else:
    # We did not find a feasible solution, we conclude that the problem is infeasible
    print("No solution found.")

Solution found. Placement time = 0.0340, Objective score maximization time = 1.0805, total = 1.1145
Objective scores: {'objective_value': -190.7054729019009, 'objective_1_value': 32.73992107636443, 'objective_2_value': 208.44539397826532, 'objective_3_value': 12, 'objective_4_value': 3}
